# PROCESAMIENTO DE DATOS

## Función general para la carga de los datos

In [37]:
def cargar_datos():
    '''
    Solicitud de las URLS de los datos y metadatos.
    Realiza dos peticiones HTTP GET:
        1. A  DATA_URL, que devuelve un JSON con los valores.
        2. A  METADATA_URL, que devuelve un JSON con información estructural.

    Returns
    -------
        - datos : Contiene los valores del conjunto de datos..
        - metadata : Contiene la información estructural de los datos (dimensiones como años y países).
    '''
   
    datos = requests.get(DATA_URL).json()  
    metadata = requests.get(METADATA_URL).json()  
    return datos, metadata


## Función general para el procesamiento de los datos

In [38]:
def procesar_datos(data_response, metadata_response):
    '''
    Procesa la respuesta de datos y metadata para extraer la información necesaria.

    Parámetros:
    ----------
    data_response : dict
        Diccionario JSON obtenido de la API que contiene los valores numéricos.
    
    metadata_response : dict
        Diccionario JSON obtenido de la API que contiene información adicional como los años disponibles
        y los países asociados a los datos.

    Returns:
    --------
        - valores (list): Lista de valores numéricos extraídos del campo "values".
        - años (list): Lista de identificadores de años extraídos de la metadata.
        - paises (dict): Diccionario que mapea IDs de país a nombres de país, obtenido de la metadata.

    Excepciones:
    -----------
    ValueError:
        Se lanza si no se encuentran las claves esperadas ("values", "years", "entities") en las estructuras 
        JSON correspondientes.
    '''
    # Extraer los valores numéricos 
    if "values" in data_response:
        valores = data_response["values"]
    else:
        raise ValueError("No se encontró la clave 'values' en los datos")

    # Extraer la lista de años disponibles
    if "dimensions" in metadata_response and "years" in metadata_response["dimensions"]:
        años = [item["id"] for item in metadata_response["dimensions"]["years"]["values"]]
    else:
        raise ValueError("No se encontró la clave 'years' en la metadata")

    # Extraer los países o entidades desde la metadata 
    if "dimensions" in metadata_response and "entities" in metadata_response["dimensions"]:
        paises = {item["id"]: item["name"] for item in metadata_response["dimensions"]["entities"]["values"]}
    else:
        raise ValueError("No se encontró la clave 'entities' en la metadata")
    
    return valores, años, paises

## Función general para la creación de DF

In [39]:
import pandas as pd

def crear_dataframe(valores, años, paises, nombre_columna):
    '''
    Crea un DataFrame de pandas combinando listas de valores, años y países.

    Esta función organiza los datos recibidos desde una API en un DataFrame estructurado,
    asignando los valores correspondientes a cada combinación de país y año. Si hay menos
    valores que combinaciones posibles, se rellena con None.

    Parámetros:
    ----------
    - valores : list
        Lista de valores numéricos, en orden secuencial.

    - años : list
        Lista de identificadores de años.

    - paises : dict
        Diccionario que mapea el ID del país al nombre del país, obtenido desde la metadata.

    - nombre_columna : str
        Nombre que se le dará a la columna de valores en el DataFrame resultante.

    Retorna:
    -------
    pd.DataFrame
        Un DataFrame con columnas: "Año", "País" y la columna con nombre "nombre_columna",
        que contiene los valores o "None" si no hay datos disponibles.
    '''
    # Crear la lista para almacenar las filas de datos
    datos_lista = []
    
    # Variable para iterar sobre los valores
    indice_valor = 0
    
    # Iterar sobre los países y años
    for id_pais, nombre_pais in paises.items():  # Iterar sobre los países
        for año in años:  # Iterar sobre los años disponibles
            if indice_valor < len(valores):
                # Si hay un valor disponible, añadirlo a la fila
                fila = {
                    "Año": año,
                    "País": nombre_pais,
                    nombre_columna: valores[indice_valor]
                }
                indice_valor += 1
            else:
                # Si no hay más valores, completar con None
                fila = {
                    "Año": año,
                    "País": nombre_pais,
                    nombre_columna: None
                }
            datos_lista.append(fila)  # Añadir la fila a la lista
    
    # Convertir la lista a un DataFrame
    df = pd.DataFrame(datos_lista)
    
    return df

## Obtención de datos del Parkinson (varibale dependiente)

In [40]:
import pandas as pd
import requests

# URLs de la API
DATA_URL = "https://api.ourworldindata.org/v1/indicators/916408.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/916408.metadata.json"
datos, metadata = cargar_datos()  
valores, años, paises = procesar_datos(datos, metadata) 
df_parkinson = crear_dataframe(valores, años, paises,"Parkinson") 

df_parkinson = df_parkinson[df_parkinson["País"] != "World"]

# Mostrar los resultados en un display
display(df_parkinson)

,Año,País,Parkinson
0,1990,Cambodia,21.533830
1,1991,Cambodia,21.483840
2,1992,Cambodia,21.502138
3,1993,Cambodia,21.607151
4,1994,Cambodia,21.767658
...,...,...,...
7291,2017,South America,101.905710
7292,2018,South America,105.467110
7293,2019,South America,108.644905
7294,2020,South America,110.542580


## Obtención de datos de la tasa contaminación del aire (varibale independiente)

In [41]:
import pandas as pd
import requests

# URLs de la API
DATA_URL = "https://api.ourworldindata.org/v1/indicators/939832.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/939832.metadata.json"
datos, metadata = cargar_datos()  
valores, años, paises = procesar_datos(datos, metadata) 
df_aire = crear_dataframe(valores, años, paises,"Contaminacion_aire") 

# Mostrar los resultados en un display
display(df_aire)

,Año,País,Contaminacion_aire
0,1990,Afghanistan,9.830483
1,1991,Afghanistan,9.412167
2,1992,Afghanistan,8.970597
3,1993,Afghanistan,8.853374
4,1994,Afghanistan,9.138245
...,...,...,...
7099,2017,Zimbabwe,2.167512
7100,2018,Zimbabwe,2.128228
7101,2019,Zimbabwe,2.164050
7102,2020,Zimbabwe,2.195074


## Obtención de datos de las muertes atribuidas a fuentes de agua inseguras, calidad de agua (varibale independiente)

In [42]:
import pandas as pd
import requests

DATA_URL = "https://api.ourworldindata.org/v1/indicators/936533.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/936533.metadata.json"
datos, metadata = cargar_datos()  
valores, años, paises = procesar_datos(datos, metadata) 
df_agua = crear_dataframe(valores, años, paises,"Muertes_agua") 

# Mostrar los resultados en un display
display(df_agua)

,Año,País,Muertes_agua
0,1990,Afghanistan,5876.4130
1,1991,Afghanistan,6145.5880
2,1992,Afghanistan,6818.0254
3,1993,Afghanistan,8395.5060
4,1994,Afghanistan,8754.6260
...,...,...,...
7291,2017,Zimbabwe,3928.3027
7292,2018,Zimbabwe,3940.3884
7293,2019,Zimbabwe,3856.9373
7294,2020,Zimbabwe,3784.6410


## Obtención de datos de la exposición al plomo (varibale independiente)

In [43]:
import pandas as pd
import requests

DATA_URL = "https://api.ourworldindata.org/v1/indicators/941463.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/941463.metadata.json"
datos, metadata = cargar_datos()  
valores, años, paises = procesar_datos(datos, metadata) 
df_plomo = crear_dataframe(valores, años, paises,"Exp_plomo") 

# Mostrar los resultados en un display
display(df_plomo)

,Año,País,Exp_plomo
0,1990,Afghanistan,2765.59400
1,1991,Afghanistan,2773.61840
2,1992,Afghanistan,2782.52610
3,1993,Afghanistan,2818.00600
4,1994,Afghanistan,2869.15920
...,...,...,...
7099,2017,Zimbabwe,759.63760
7100,2018,Zimbabwe,755.67847
7101,2019,Zimbabwe,749.21820
7102,2020,Zimbabwe,739.96454


## Obtención de datos del uso de pesticidas (varibale independiente)

In [44]:
def construir_dataframe_pesticidas(datos, metadata):
    '''
    Construye un DataFrame con los valores de uso de pesticidas por país y año a partir
    de la respuesta de una API y su metadata asociada.

    Esta función toma los datos crudos de una fuente JSON, extrae los valores, años y países,
    y los organiza en un DataFrame con todas las combinaciones posibles de país y año.
    Si no hay datos disponibles para alguna combinación, se completa con NaN.

    Parámetros:
    ----------
    - datos : dict
        Diccionario JSON que contiene los valores principales bajo las claves "values",
        "entities" (IDs de países) y "years".

    - metadata : dict
        Diccionario JSON con información adicional sobre las entidades (países) y los años
        disponibles, utilizado para mapear los IDs a nombres reales.

    Retorna:
    -------
        DataFrame con columnas: "Año", "País" y "Pesticidas", donde cada fila representa
        una combinación de país y año con su valor correspondiente (o NaN si no hay dato).
    '''
    # Extraer arrays principales
    valores = datos["values"]
    entidades = datos["entities"]
    anios = datos["years"]

    # Crear diccionario ID -> Nombre de país
    id_to_country = {
        ent["id"]: ent["name"]
        for ent in metadata["dimensions"]["entities"]["values"]
    }

    # Lista completa de años del metadata
    lista_anios = [y["id"] for y in metadata["dimensions"]["years"]["values"]]

    # Lista completa de países (nombres)
    lista_paises = [ent["name"] for ent in metadata["dimensions"]["entities"]["values"]]

    # Crear todos los pares posibles País × Año
    df_base = pd.DataFrame(
        [(pais, anio) for pais in lista_paises for anio in lista_anios],
        columns=["País", "Año"]
    )

    # Crear DataFrame con datos reales desde el JSON
    registros = []
    for i in range(len(valores)):
        pais_id = entidades[i]
        anio = anios[i]
        valor = valores[i]
        pais = id_to_country.get(pais_id)
        if pais:
            registros.append((anio, pais, valor))  # Año primero

    df_valores = pd.DataFrame(registros, columns=["Año", "País", "Pesticidas"])

    # Merge con la tabla base (left join para conservar todos los países y años)
    df_final = pd.merge(df_base, df_valores, on=["Año", "País"], how="left")

    # Reordenar columnas
    df_final = df_final[["Año", "País", "Pesticidas"]]

    return df_final

In [45]:
DATA_URL = "https://api.ourworldindata.org/v1/indicators/1016584.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/1016584.metadata.json"
datos, metadata = cargar_datos()  
df_pepticidas = construir_dataframe_pesticidas(datos, metadata)
display(df_pepticidas)


,Año,País,Pesticidas
0,1990,Africa (FAO),73504.69
1,1991,Africa (FAO),69490.83
2,1992,Africa (FAO),59769.13
3,1993,Africa (FAO),53768.65
4,1994,Africa (FAO),58312.42
...,...,...,...
8245,2018,Zimbabwe,2185.07
8246,2019,Zimbabwe,2185.07
8247,2020,Zimbabwe,2185.07
8248,2021,Zimbabwe,2185.07


## Obtención de datos de la cantidad de precipitaciones (varibale independiente)

In [46]:
def crear_dataframe_p(valores, años, paises, nombre_columna):
    '''
    Crea un DataFrame para los datos de las precipitaciones combinando países y años, asignando valores de forma secuencial.

    Para cada combinación (Año, País), se asigna un valor de la lista "valores". Si no hay
    suficientes valores para todas las combinaciones, se rellena con None.

    Parameters
    ----------
    - valores : list
        Lista de valores numéricos u observaciones.
    - años : list
        Lista de años disponibles.
    - paises : dict
        Diccionario con ID de país como claves y nombres de país como valores.
    - nombre_columna : str
        Nombre que se usará para la columna de los valores en el DataFrame.

    Returns
    -------
        DataFrame con columnas: "Año", "País" y "Precipitaciones", combinando todos los países
        y años con los valores correspondientes.
    '''
    datos_lista = []
    indice_valor = 0
    
    for año in años:
        for id_pais, nombre_pais in paises.items():
            if indice_valor < len(valores):
                fila = {
                    "Año": año,
                    "País": nombre_pais,
                    nombre_columna: valores[indice_valor]
                }
                indice_valor += 1
            else:
                fila = {
                    "Año": año,
                    "País": nombre_pais,
                    nombre_columna: None
                }
            datos_lista.append(fila)
    
    df = pd.DataFrame(datos_lista)
    return df

In [47]:
import pandas as pd
import requests

DATA_URL = "https://api.ourworldindata.org/v1/indicators/1005182.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/1005182.metadata.json"
datos, metadata = cargar_datos()  
valores, años, paises = procesar_datos(datos, metadata) 
df_precipitaciones = crear_dataframe_p(valores, años, paises,"Precipitaciones") 

# Mostrar los resultados en un display
display(df_precipitaciones)



,Año,País,Precipitaciones
0,1940,Afghanistan,217.678160
1,1940,Albania,1388.520100
2,1940,Algeria,45.775917
3,1940,American Samoa,2582.801800
4,1940,Andorra,899.675350
...,...,...,...
16570,2024,World,1073.184600
16571,2024,Yemen,147.840290
16572,2024,Zambia,795.214360
16573,2024,Zimbabwe,421.734000


# ELIMINAR NULOS Y GUARDARLO EN .CSV

In [48]:
import pandas as pd

import os

def limpiar_y_guardar_csv(df, nombre_archivo):
    '''
    Elimina filas con valores nulos de un DataFrame y guarda el resultado en un archivo CSV
    dentro de la ruta: Resultados/Archivos_csv/Datos.

    Parameters
    ----------
    - df : pd.DataFrame
        DataFrame original que puede contener valores nulos.
    - nombre_archivo : str
        Nombre del archivo CSV (sin ruta) donde se guardarán los datos limpios.

    Returns
    -------
    - DataFrame resultante después de eliminar las filas con valores nulos.
    '''
    # Eliminar filas con valores nulos
    df_limpio = df.dropna()

    # Crear ruta destino
    ruta_destino = os.path.join("..","Resultados", "Archivos_csv", "Datos")
    os.makedirs(ruta_destino, exist_ok=True)

    # Ruta completa al archivo
    ruta_completa = os.path.join(ruta_destino, nombre_archivo)

    # Guardar CSV
    df_limpio.to_csv(ruta_completa, index=False, encoding="utf-8")

    print(f"Datos guardados en '{ruta_completa}' sin valores nulos.")
    
    return df_limpio


## Parkinson

In [49]:
limpiar_y_guardar_csv(df_parkinson,"Datos_Parkinson.csv")

Datos guardados en '..\Resultados\Archivos_csv\Datos\Datos_Parkinson.csv' sin valores nulos.


,Año,País,Parkinson
0,1990,Cambodia,21.533830
1,1991,Cambodia,21.483840
2,1992,Cambodia,21.502138
3,1993,Cambodia,21.607151
4,1994,Cambodia,21.767658
...,...,...,...
7291,2017,South America,101.905710
7292,2018,South America,105.467110
7293,2019,South America,108.644905
7294,2020,South America,110.542580


## Contaminación de aire

In [50]:
limpiar_y_guardar_csv(df_aire,"Datos_contaminación_aire.csv")

Datos guardados en '..\Resultados\Archivos_csv\Datos\Datos_contaminación_aire.csv' sin valores nulos.


,Año,País,Contaminacion_aire
0,1990,Afghanistan,9.830483
1,1991,Afghanistan,9.412167
2,1992,Afghanistan,8.970597
3,1993,Afghanistan,8.853374
4,1994,Afghanistan,9.138245
...,...,...,...
7099,2017,Zimbabwe,2.167512
7100,2018,Zimbabwe,2.128228
7101,2019,Zimbabwe,2.164050
7102,2020,Zimbabwe,2.195074


## Muertes atribuidas a fuentes de agua inseguras

In [51]:
limpiar_y_guardar_csv(df_agua,"Datos_muertes_agua.csv")

Datos guardados en '..\Resultados\Archivos_csv\Datos\Datos_muertes_agua.csv' sin valores nulos.


,Año,País,Muertes_agua
0,1990,Afghanistan,5876.4130
1,1991,Afghanistan,6145.5880
2,1992,Afghanistan,6818.0254
3,1993,Afghanistan,8395.5060
4,1994,Afghanistan,8754.6260
...,...,...,...
7291,2017,Zimbabwe,3928.3027
7292,2018,Zimbabwe,3940.3884
7293,2019,Zimbabwe,3856.9373
7294,2020,Zimbabwe,3784.6410


## Exposicion al plomo

In [52]:
limpiar_y_guardar_csv(df_plomo,"Datos_exp_plomo.csv")

Datos guardados en '..\Resultados\Archivos_csv\Datos\Datos_exp_plomo.csv' sin valores nulos.


,Año,País,Exp_plomo
0,1990,Afghanistan,2765.59400
1,1991,Afghanistan,2773.61840
2,1992,Afghanistan,2782.52610
3,1993,Afghanistan,2818.00600
4,1994,Afghanistan,2869.15920
...,...,...,...
7099,2017,Zimbabwe,759.63760
7100,2018,Zimbabwe,755.67847
7101,2019,Zimbabwe,749.21820
7102,2020,Zimbabwe,739.96454


## Uso de pepticidas

In [53]:
limpiar_y_guardar_csv(df_pepticidas,"Datos_uso_pepticidas.csv")

Datos guardados en '..\Resultados\Archivos_csv\Datos\Datos_uso_pepticidas.csv' sin valores nulos.


,Año,País,Pesticidas
0,1990,Africa (FAO),73504.69
1,1991,Africa (FAO),69490.83
2,1992,Africa (FAO),59769.13
3,1993,Africa (FAO),53768.65
4,1994,Africa (FAO),58312.42
...,...,...,...
8245,2018,Zimbabwe,2185.07
8246,2019,Zimbabwe,2185.07
8247,2020,Zimbabwe,2185.07
8248,2021,Zimbabwe,2185.07


## Precipitaciones

In [54]:
limpiar_y_guardar_csv(df_precipitaciones,"Datos_precipitaciones.csv")

Datos guardados en '..\Resultados\Archivos_csv\Datos\Datos_precipitaciones.csv' sin valores nulos.


,Año,País,Precipitaciones
0,1940,Afghanistan,217.678160
1,1940,Albania,1388.520100
2,1940,Algeria,45.775917
3,1940,American Samoa,2582.801800
4,1940,Andorra,899.675350
...,...,...,...
16570,2024,World,1073.184600
16571,2024,Yemen,147.840290
16572,2024,Zambia,795.214360
16573,2024,Zimbabwe,421.734000


## CARGA DE CSV NECESARIOS PARA LA CONSTRUCCIÓN DE TABLAS FINALES

In [55]:
import os
import pandas as pd

def cargar_csv(nombre_archivo):
    """
    Carga un archivo CSV desde la ruta 'Resultados/Archivos_csv/Datos' en un DataFrame de pandas.

    Parámetros
    ----------
    nombre_archivo : str
        Nombre del archivo CSV a cargar.

    Retorna
    -------
    pd.DataFrame
        DataFrame con los datos cargados del archivo CSV.
    """
    # Construir la ruta completa
    ruta_archivo = os.path.join("..","Resultados", "Archivos_csv", "Datos", nombre_archivo)
    
    # Leer el CSV
    df = pd.read_csv(ruta_archivo, encoding="utf-8")
    print(f"Archivo '{ruta_archivo}' cargado correctamente.")
    
    return df


In [56]:
df_parkinson = cargar_csv("Datos_Parkinson.csv")
df_contaminacion = cargar_csv("Datos_contaminación_aire.csv")  # Tasa de contaminación
df_calidad_agua = cargar_csv("Datos_muertes_agua.csv")
df_plomo = cargar_csv("Datos_exp_plomo.csv")  # Tasa de carga de enfermedad por exposición al plomo
df_pesticidas = cargar_csv("Datos_uso_pepticidas.csv")  # Uso de pesticidas en toneladas
df_precipitaciones = cargar_csv("Datos_precipitaciones.csv")  # Precipitaciones en mm

Archivo '..\Resultados\Archivos_csv\Datos\Datos_Parkinson.csv' cargado correctamente.
Archivo '..\Resultados\Archivos_csv\Datos\Datos_contaminación_aire.csv' cargado correctamente.
Archivo '..\Resultados\Archivos_csv\Datos\Datos_muertes_agua.csv' cargado correctamente.
Archivo '..\Resultados\Archivos_csv\Datos\Datos_exp_plomo.csv' cargado correctamente.
Archivo '..\Resultados\Archivos_csv\Datos\Datos_uso_pepticidas.csv' cargado correctamente.
Archivo '..\Resultados\Archivos_csv\Datos\Datos_precipitaciones.csv' cargado correctamente.


## Unión de Dataframes

In [57]:
# Unir todos los DataFrames por 'Año' y 'País'
df = df_parkinson.merge(df_contaminacion, on=['Año', 'País'], how='left')
df = df.merge(df_plomo, on=['Año', 'País'], how='left')
df = df.merge(df_calidad_agua, on=["Año","País"], how='left')
df = df.merge(df_pesticidas, on=['Año', 'País'], how='left')
df_final = df.merge(df_precipitaciones, on=['Año', 'País'], how='left')

# Mostrar los primeros datos
display(df_final)

,Año,País,Parkinson,Contaminacion_aire,Exp_plomo,Muertes_agua,Pesticidas,Precipitaciones
0,1990,Cambodia,21.533830,0.770228,799.84644,5680.9985,480.55,1810.3473
1,1991,Cambodia,21.483840,0.867021,798.64690,5618.9200,480.55,1720.9519
2,1992,Cambodia,21.502138,0.876114,799.89874,5418.9050,480.55,1572.4829
3,1993,Cambodia,21.607151,0.992484,804.59204,5273.5493,480.55,1765.2129
4,1994,Cambodia,21.767658,0.902714,812.10320,5176.5522,480.55,1684.9478
...,...,...,...,...,...,...,...,...
7259,2017,South America,101.905710,NaN,NaN,5844.3230,NaN,NaN
7260,2018,South America,105.467110,NaN,NaN,5681.7460,NaN,NaN
7261,2019,South America,108.644905,NaN,NaN,5672.4670,NaN,NaN
7262,2020,South America,110.542580,NaN,NaN,5413.5270,NaN,NaN


## Cálculo de nulos

In [58]:
# Calcular el porcentaje de valores NaN en cada columna
nan_percentage = df_final.isnull().mean() * 100

# Mostrar el porcentaje de valores NaN por columna
print(nan_percentage)

Año                    0.000000
País                   0.000000
Parkinson              0.000000
Contaminacion_aire     2.643172
Exp_plomo              2.643172
Muertes_agua           0.000000
Pesticidas            18.020374
Precipitaciones       19.823789
dtype: float64


In [59]:
# Calcular el porcentaje total de valores nulos en el dataframe
total_nan_percentage = df_final.isnull().sum().sum() / df_final.size * 100

# Mostrar el resultado
print(f"Porcentaje total de valores nulos: {total_nan_percentage:.2f}%")

Porcentaje total de valores nulos: 5.39%


### Elimincación de nulos

In [60]:
df_final_no_nulos  = df_final.dropna()
display(df_final_no_nulos)

,Año,País,Parkinson,Contaminacion_aire,Exp_plomo,Muertes_agua,Pesticidas,Precipitaciones
0,1990,Cambodia,21.533830,0.770228,799.84644,5680.9985,480.55,1810.34730
1,1991,Cambodia,21.483840,0.867021,798.64690,5618.9200,480.55,1720.95190
2,1992,Cambodia,21.502138,0.876114,799.89874,5418.9050,480.55,1572.48290
3,1993,Cambodia,21.607151,0.992484,804.59204,5273.5493,480.55,1765.21290
4,1994,Cambodia,21.767658,0.902714,812.10320,5176.5522,480.55,1684.94780
...,...,...,...,...,...,...,...,...
6747,2017,Sudan,33.183840,2.207330,1261.99870,2538.9585,2379.65,155.55054
6748,2018,Sudan,33.985485,2.483574,1248.51800,1889.6714,5486.85,193.85168
6749,2019,Sudan,34.539875,2.573782,1235.76650,1719.6345,4082.53,254.14426
6750,2020,Sudan,33.731514,2.687866,1224.61080,1554.0481,4082.53,191.68684


## Construcción de la tabla para los modelos

In [61]:
ultimo_año = df_final_no_nulos ["Año"].max()  # Encuentra el último año
df_sin_año = df_final_no_nulos [df_final_no_nulos ["Año"] != ultimo_año]
df_final_modelo = df_final_no_nulos.drop(columns=['Año', 'País'])
display(df_final_modelo)

,Parkinson,Contaminacion_aire,Exp_plomo,Muertes_agua,Pesticidas,Precipitaciones
0,21.533830,0.770228,799.84644,5680.9985,480.55,1810.34730
1,21.483840,0.867021,798.64690,5618.9200,480.55,1720.95190
2,21.502138,0.876114,799.89874,5418.9050,480.55,1572.48290
3,21.607151,0.992484,804.59204,5273.5493,480.55,1765.21290
4,21.767658,0.902714,812.10320,5176.5522,480.55,1684.94780
...,...,...,...,...,...,...
6747,33.183840,2.207330,1261.99870,2538.9585,2379.65,155.55054
6748,33.985485,2.483574,1248.51800,1889.6714,5486.85,193.85168
6749,34.539875,2.573782,1235.76650,1719.6345,4082.53,254.14426
6750,33.731514,2.687866,1224.61080,1554.0481,4082.53,191.68684


In [62]:
ruta_destino = os.path.join("..", "Resultados", "Archivos_csv", "Datos")
# Construir la ruta completa al archivo
ruta_archivo = os.path.join(ruta_destino, "Tabla_modelos.csv")
# Guardar el DataFrame en esa ruta
df_final_modelo.to_csv(ruta_archivo, index=False, encoding="utf-8")
print(f"Archivo guardado en: {ruta_archivo}")


Archivo guardado en: ..\Resultados\Archivos_csv\Datos\Tabla_modelos.csv


## Construcción de tabla para entrenar

In [63]:
# Suponiendo que tu DataFrame se llama df y la columna de años es "año"
primer_año = df_final_no_nulos["Año"].min()
penultimo_año = sorted(df_final_no_nulos["Año"].unique())[-2]  # Encuentra el penúltimo año

# Filtrar el DataFrame
df_filtrado = df_final_no_nulos[(df_final_no_nulos["Año"] >= primer_año) & (df_final_no_nulos["Año"] <= penultimo_año)]
df_filtrado_modelo = df_filtrado.drop(columns=['Año'])
display(df_filtrado_modelo)

,País,Parkinson,Contaminacion_aire,Exp_plomo,Muertes_agua,Pesticidas,Precipitaciones
0,Cambodia,21.533830,0.770228,799.84644,5680.9985,480.55,1810.34730
1,Cambodia,21.483840,0.867021,798.64690,5618.9200,480.55,1720.95190
2,Cambodia,21.502138,0.876114,799.89874,5418.9050,480.55,1572.48290
3,Cambodia,21.607151,0.992484,804.59204,5273.5493,480.55,1765.21290
4,Cambodia,21.767658,0.902714,812.10320,5176.5522,480.55,1684.94780
...,...,...,...,...,...,...,...
6746,Sudan,32.230840,2.266018,1279.99110,2360.7010,3254.33,170.96439
6747,Sudan,33.183840,2.207330,1261.99870,2538.9585,2379.65,155.55054
6748,Sudan,33.985485,2.483574,1248.51800,1889.6714,5486.85,193.85168
6749,Sudan,34.539875,2.573782,1235.76650,1719.6345,4082.53,254.14426


In [65]:
ruta_destino = os.path.join("..", "Resultados", "Archivos_csv", "Datos")
ruta_archivo = os.path.join(ruta_destino, "Tabla_filtrada.csv")
# Guardar el DataFrame en esa ruta
df_filtrado_modelo.to_csv(ruta_archivo, index=False, encoding="utf-8")
print(f"Archivo guardado en: {ruta_archivo}")

Archivo guardado en: ..\Resultados\Archivos_csv\Datos\Tabla_filtrada.csv


## Construcción de tabla para predecir

In [32]:
ultimo_año = df_final_no_nulos ["Año"].max()  # Encuentra el último año

df_ultimo_año = df_final_no_nulos [df_final_no_nulos ["Año"] == ultimo_año]  # Filtra solo ese año
df_predecir = df_ultimo_año.drop(columns = ["Año","Parkinson"])
display(df_predecir)

,País,Contaminacion_aire,Exp_plomo,Muertes_agua,Pesticidas,Precipitaciones
31,Cambodia,2.579782,665.62980,793.334800,17390.37,1828.92940
63,China,7.186636,458.91144,2380.870600,233876.05,827.30980
127,Indonesia,2.977838,639.60520,33702.586000,283297.12,3506.87600
159,Laos,4.151983,852.68396,571.478700,187.31,1848.15220
191,Malaysia,2.090159,301.38852,532.311800,45671.00,3160.46440
...,...,...,...,...,...,...
5919,Sierra Leone,2.022849,681.09000,2544.818800,360.95,2654.27780
5951,Togo,4.086113,629.94480,2289.433000,1425.00,1074.55210
6239,Cook Islands,0.588149,131.94354,0.364360,14.93,1881.73960
6271,Greenland,1.560168,106.91559,0.048419,8.61,475.92148


In [66]:
ruta_archivo = os.path.join(ruta_destino, "Tabla_para_predecir.csv")
# Guardar el DataFrame en esa ruta
df_predecir.to_csv(ruta_archivo, index=False, encoding="utf-8")
print(f"Archivo guardado en: {ruta_archivo}")

Archivo guardado en: ..\Resultados\Archivos_csv\Datos\Tabla_para_predecir.csv


## Contrucción de tabla para la comparación posterior (datos parkinson 2021)

In [35]:
df_parkinson_2021 = df_final_no_nulos[df_final_no_nulos['Año'] == 2021].round(2)

# Mostramos el DataFrame resultante
display(df_parkinson_2021[['País', 'Parkinson']])

,País,Parkinson
31,Cambodia,47.48
63,China,356.85
127,Indonesia,58.05
159,Laos,40.36
191,Malaysia,75.38
...,...,...
5919,Sierra Leone,23.91
5951,Togo,24.02
6239,Cook Islands,153.09
6271,Greenland,123.93


In [67]:
ruta_archivo = os.path.join(ruta_destino, "Tabla_para_comparar.csv")
# Guardar el DataFrame en esa ruta
df_parkinson_2021.to_csv(ruta_archivo, index=False, encoding="utf-8")
print(f"Archivo guardado en: {ruta_archivo}")

Archivo guardado en: ..\Resultados\Archivos_csv\Datos\Tabla_para_comparar.csv
